# Equation of state (EOS)

For the source code, see [eos](https://workgraph-collections.readthedocs.io/en/latest/ase/espresso/module.html#workgraph_collections.ase.espresso.eos.eos_workgraph).

## Visualizing the WorkGraph Builder


In [1]:
from workgraph_collections.ase.espresso.eos import eos_workgraph

task = eos_workgraph._TaskCls()
task.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.eos import eos_workgraph
from aiida import load_profile

load_profile()

wg = eos_workgraph()
# wg.to_html()
# wg

## Example: Silicon

### Prepare the inputs and submit the workflow


In [4]:
from ase.build import bulk
from ase import Atoms
from aiida import load_profile
from workgraph_collections.ase.espresso.eos import eos_workgraph
from aiida_pythonjob.config import config

load_profile()

config["allow_pickle"] = True

# create input structure
atoms = bulk("Si")

metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
input_data = {
    "control": {"calculation": "scf"},
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold",},
}
#------------------------- Set the inputs -------------------------
wg = eos_workgraph(atoms=atoms,
                   computer="localhost",
                   scales=[0.95, 0.98, 1.0, 1.02, 1.05],
                   command="mpirun -np 2 pw.x",
                   pseudopotentials=pseudopotentials,
                   pseudo_dir=pseudo_dir,
                   input_data=input_data,
                   kpts=[4, 4, 4],
                   metadata=metadata)
#------------------------- Submit the calculation -------------------
wg.run()
# wg.submit(wait=True, timeout=200)



03/27/2025 09:59:12 PM <1595705> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [63172|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator1
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[63172|WorkGraphEngine|continue_workgraph]: tasks ready to run: pw_calculator1
03/27/2025 09:59:13 PM <1595705> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [63172|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 63178
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[63172|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 63178
03/27/2025 09:59:26 PM <1595705> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [63172|WorkGraphEngine|update_task_state]: Task: pw_calculator1, type: PythonJob, finished.
REPORT:aiida.orm.nodes.process.workflow.workchain.WorkChainNode:[63172|WorkGraphEngine|update_task_state]: Task: pw_calculator1, type: PythonJob, finished

In [5]:
#------------------------- Print the output -------------------------
print('\nResult: \nB: {B}\nv0: {v0}\ne0: {e0}\nv0: {v0}'.format(**wg.tasks[-1].outputs.result.value))



Result: 
B: 88.541858293157
v0: 41.224378688729
e0: -277.89646661704
v0: 41.224378688729
